# Cannalchemy Data Exploration

Exploratory analysis of the data foundation: strains, terpenes, effects, and molecular interactions.

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from cannalchemy.data.schema import init_db
from cannalchemy.data.graph import build_knowledge_graph, get_strain_profile, get_molecule_pathways

sns.set_theme(style='whitegrid', palette='husl')
plt.rcParams['figure.figsize'] = (12, 6)

DB_PATH = '../data/processed/cannalchemy.db'
conn = sqlite3.connect(DB_PATH)
print('Connected to', DB_PATH)

## 1. Data Quality Overview

In [ ]:
# Overall counts
tables = ['strains', 'molecules', 'receptors', 'binding_affinities',
          'strain_compositions', 'effects', 'effect_reports']
counts = {}
for t in tables:
    row = conn.execute(f'SELECT COUNT(*) FROM {t}').fetchone()
    counts[t] = row[0]

df_counts = pd.DataFrame(list(counts.items()), columns=['Table', 'Rows'])
display(df_counts)

# Strain type distribution
df_types = pd.read_sql('SELECT strain_type, COUNT(*) as count FROM strains GROUP BY strain_type', conn)
print('\nStrain types:')
display(df_types)

## 2. Terpene Frequency Distribution

In [ ]:
# Which terpenes appear most frequently across strains?
df_terp_freq = pd.read_sql("""
    SELECT m.name as terpene, COUNT(DISTINCT sc.strain_id) as strain_count,
           AVG(sc.percentage) as avg_pct, MAX(sc.percentage) as max_pct
    FROM strain_compositions sc
    JOIN molecules m ON sc.molecule_id = m.id
    WHERE m.molecule_type = 'terpene'
    GROUP BY m.name
    ORDER BY strain_count DESC
""", conn)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Frequency bar chart
ax1.barh(df_terp_freq['terpene'], df_terp_freq['strain_count'])
ax1.set_xlabel('Number of Strains')
ax1.set_title('Terpene Frequency Across Strains')
ax1.invert_yaxis()

# Average percentage
ax2.barh(df_terp_freq['terpene'], df_terp_freq['avg_pct'])
ax2.set_xlabel('Average Percentage (%)')
ax2.set_title('Average Terpene Concentration')
ax2.invert_yaxis()

plt.tight_layout()
plt.savefig('../data/processed/terpene_frequency.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Terpene-Effect Correlation Matrix

In [ ]:
# Build a strain x terpene matrix
df_comp = pd.read_sql("""
    SELECT sc.strain_id, m.name as molecule, sc.percentage
    FROM strain_compositions sc
    JOIN molecules m ON sc.molecule_id = m.id
    WHERE m.molecule_type = 'terpene'
""", conn)
terpene_matrix = df_comp.pivot_table(index='strain_id', columns='molecule', values='percentage', fill_value=0)

# Build a strain x effect binary matrix
df_eff = pd.read_sql("""
    SELECT er.strain_id, e.name as effect
    FROM effect_reports er
    JOIN effects e ON er.effect_id = e.id
    WHERE e.category = 'positive'
""", conn)
effect_matrix = df_eff.pivot_table(index='strain_id', columns='effect', aggfunc='size', fill_value=0)
effect_matrix = (effect_matrix > 0).astype(int)

# Align on shared strain_ids
shared = terpene_matrix.index.intersection(effect_matrix.index)
print(f'Strains with both terpene and effect data: {len(shared)}')

if len(shared) > 100:
    # Compute correlation between terpene levels and effect presence
    combined = pd.concat([terpene_matrix.loc[shared], effect_matrix.loc[shared]], axis=1)
    
    # Select top 10 effects by frequency
    top_effects = effect_matrix.loc[shared].sum().nlargest(10).index.tolist()
    top_terpenes = terpene_matrix.columns.tolist()
    
    corr = combined[top_terpenes + top_effects].corr().loc[top_terpenes, top_effects]
    
    plt.figure(figsize=(14, 10))
    sns.heatmap(corr, annot=True, fmt='.2f', cmap='RdBu_r', center=0, vmin=-0.3, vmax=0.3)
    plt.title('Terpene-Effect Correlation (top 10 effects)')
    plt.tight_layout()
    plt.savefig('../data/processed/terpene_effect_correlation.png', dpi=150, bbox_inches='tight')
    plt.show()
else:
    print('Not enough shared data for correlation analysis')

## 4. Knowledge Graph Visualization

In [ ]:
# Build the knowledge graph
G = build_knowledge_graph(conn)
print(f'Graph: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges')

# Extract just the molecular interaction subgraph (molecules + receptors + bindings)
mol_nodes = [n for n, d in G.nodes(data=True) if d.get('node_type') in ('molecule', 'receptor')]
binding_edges = [(u, v) for u, v, d in G.edges(data=True) if d.get('edge_type') == 'binds_to']

# Build subgraph
H = G.subgraph(set(n for e in binding_edges for n in e)).copy()
print(f'Binding subgraph: {H.number_of_nodes()} nodes, {H.number_of_edges()} edges')

# Color by node type
color_map = []
for node in H.nodes():
    ntype = H.nodes[node].get('node_type')
    if ntype == 'molecule':
        mtype = H.nodes[node].get('molecule_type', '')
        color_map.append('#2ecc71' if mtype == 'terpene' else '#e74c3c')
    else:
        color_map.append('#3498db')

labels = {n: H.nodes[n]['name'] for n in H.nodes()}

plt.figure(figsize=(16, 12))
pos = nx.spring_layout(H, k=2, seed=42)
nx.draw_networkx(H, pos, labels=labels, node_color=color_map,
                 node_size=800, font_size=8, arrows=True, alpha=0.9,
                 edge_color='#bdc3c7', width=1.5)

# Legend
import matplotlib.patches as mpatches
legend = [mpatches.Patch(color='#2ecc71', label='Terpene'),
          mpatches.Patch(color='#e74c3c', label='Cannabinoid'),
          mpatches.Patch(color='#3498db', label='Receptor')]
plt.legend(handles=legend, loc='upper left', fontsize=12)
plt.title('Molecular Interaction Network (binds_to edges)', fontsize=14)
plt.axis('off')
plt.tight_layout()
plt.savefig('../data/processed/knowledge_graph.png', dpi=150, bbox_inches='tight')
plt.show()

## 5. Strain Profile Example

In [ ]:
# Look up a well-known strain
profile = get_strain_profile(G, 'Blue Dream')
if profile:
    print(f"Strain: {profile['name']} ({profile['type']})")
    print(f"\nCompositions ({len(profile['compositions'])}):\n")
    for c in profile['compositions'][:10]:
        print(f"  {c['molecule']:15s} {c['percentage']:6.2f}%  ({c['type']})")
    print(f"\nTop Effects ({len(profile['effects'])}):\n")
    for e in profile['effects'][:10]:
        print(f"  {e['effect']:15s} ({e['category']})")
    print(f"\nMolecular Pathways ({len(profile['pathways'])}):\n")
    for p in profile['pathways']:
        ki = f"{p['ki_nm']:.1f} nM" if p['ki_nm'] else 'N/A'
        print(f"  {p['molecule']:15s} → {p['receptor']:10s} Ki={ki:>12s}  ({p['action_type']})")
else:
    print('Blue Dream not found in graph')

In [ ]:
conn.close()
print('Done! Database connection closed.')